# Fake News through Stance Detection

## Problem

Detecting whether an article is Fake News using Stance Detection. Stance detection takes a headline and body text of an article and classifies the stance of the body text relative to headline. 

In [20]:
import pandas as pd 

import os
import re
import nltk
from sklearn import feature_extraction
from tqdm import tqdm


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

In [3]:
bodies = pd.read_csv('./train_bodies.csv')
stances = pd.read_csv('./train_stances.csv')


## EDA

In [5]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [6]:
stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [7]:
stances[stances['Body ID'] == 712]

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1787,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss
3974,Mexico police find mass grave near site 43 stu...,712,unrelated
4936,Mexico Says Missing Students Not Found In Firs...,712,unrelated
5210,New iOS 8 bug can delete all of your iCloud do...,712,unrelated
5863,Return of the Mac: Seth Rogen in talks to star...,712,discuss
6199,Seth Rogen Is Woz,712,discuss
6756,Mexico finds 4 more graves at site of suspecte...,712,unrelated
7526,Are missing students in mass graves found near...,712,unrelated
9003,Mexico prosecutor: Students not in 1st mass gr...,712,unrelated


In [8]:
stances['Body ID'].value_counts()

1921    187
1948    175
40      172
524     171
1549    166
304     154
1385    151
125     145
2367    143
220     141
1438    141
195     140
2296    136
35      131
1786    131
1883    131
2520    127
1034    127
2252    126
1574    125
2307    125
527     125
2175    124
1627    123
2404    123
1289    122
2115    121
2096    120
1040    118
1893    117
       ... 
907       1
370       1
210       1
146       1
114       1
1542      1
63        1
76        1
390       1
515       1
193       1
464       1
355       1
323       1
624       1
282       1
18        1
797       1
701       1
362       1
2311      1
6         1
915       1
70        1
151       1
376       1
140       1
307       1
1066      1
59        1
Name: Body ID, Length: 1683, dtype: int64

In [9]:
stances['Stance'].unique()

array(['unrelated', 'agree', 'disagree', 'discuss'], dtype=object)

In [10]:
print('Bodies shape:', bodies.shape)
print('Stances shape:', stances.shape)


Bodies shape: (1683, 2)
Stances shape: (49972, 3)


In [11]:
stances['Stance'].value_counts()/stances.shape[0]

unrelated    0.731310
discuss      0.178280
agree        0.073601
disagree     0.016809
Name: Stance, dtype: float64

In [12]:
#why are stances rows much larger

In [13]:
bodies.isnull().sum()

Body ID        0
articleBody    0
dtype: int64

In [14]:
stances.isnull().sum()

Headline    0
Body ID     0
Stance      0
dtype: int64

In [15]:
bodies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 2 columns):
Body ID        1683 non-null int64
articleBody    1683 non-null object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [16]:
stances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49972 entries, 0 to 49971
Data columns (total 3 columns):
Headline    49972 non-null object
Body ID     49972 non-null int64
Stance      49972 non-null object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
print("Total stances: " + str(len(stances.Stance)))
print("Total article bodies: " + str(len(bodies.articleBody)))

Total stances: 49972
Total article bodies: 1683


In [4]:
#merge on bodyID 

df_all = stances.merge(bodies, on = 'Body ID')
#df_all = df_all.drop('Stance', axis =1)
df_all.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss,Danny Boyle is directing the untitled film\n\n...
2,Mexico police find mass grave near site 43 stu...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
3,Mexico Says Missing Students Not Found In Firs...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
4,New iOS 8 bug can delete all of your iCloud do...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [19]:
df_all.isnull().sum()

Headline       0
Body ID        0
Stance         0
articleBody    0
dtype: int64

In [20]:
df_all.shape

(49972, 4)

## Baseline

In [21]:
stances['Stance'].value_counts(normalize=True)

unrelated    0.731310
discuss      0.178280
agree        0.073601
disagree     0.016809
Name: Stance, dtype: float64

# NLP

## Count Vectorizer

In [22]:
#body
X_train = df_all[['articleBody']]
y_train = stances['Stance']
X_test = df_all[['articleBody']]
y_test = stances['Stance']

cvec_body = CountVectorizer(stop_words = 'english')

train_body_cvec = cvec_body.fit_transform(X_train['articleBody'])
test_body_cvec = cvec_body.transform(X_test['articleBody'])

lr = LogisticRegression()
lr.fit(train_body_cvec, y_train)
lr.score(test_body_cvec, y_test)

0.7320899703834147

In [23]:
#headline
X_train = df_all[['Headline']]
y_train = stances['Stance']
X_test = df_all[['Headline']]
y_test = stances['Stance']

cvec_headline= CountVectorizer(stop_words = 'english')

train_headline_cvec = cvec_headline.fit_transform(X_train['Headline'])
test_headline_cvec = cvec_headline.transform(X_test['Headline'])

lr = LogisticRegression()
lr.fit(train_headline_cvec, y_train)
lr.score(test_headline_cvec, y_test)

0.7313495557512207

In [24]:
#combine headline and body cvec and that will be x_train
#fit model on X_train 
#if model doesnt do well do truncated svd then cosine similarity between bodies and headline which will become a feature 


In [25]:
#do i still need these 
#df  = pd.DataFrame(X_test_count.todense(),
                   #columns=cvec.get_feature_names(), index = [bodies.articleBody]) 
#df.head()


In [26]:
#df.sort_index(axis = 0, ascending=False).head()

In [27]:
#df.sum().sort_values(ascending=False).head(10)

In [28]:
#df = df.reindex_axis(df.sum().sort_values(ascending=False).index, axis =1)
#df

# Hashing Vectorizer

In [29]:
#body
X_train = df_all[['articleBody']]
y_train = stances['Stance']
X_test = df_all[['articleBody']]
y_test = stances['Stance']

In [30]:
hvec_body = HashingVectorizer(stop_words = 'english')

train_body_hvec = hvec_body.fit_transform(X_train['articleBody'])
test_body_hvec = hvec_body.transform(X_test['articleBody'])

lr = LogisticRegression()
lr.fit(train_body_hvec, y_train)
lr.score(test_body_hvec, y_test)

0.7313095333386697

In [31]:
#headline
X_train = df_all[['Headline']]
y_train = stances['Stance']
X_test = df_all[['Headline']]
y_test = stances['Stance']

In [32]:
hvec_headline = HashingVectorizer(stop_words = 'english')

train_headline_hvec = hvec_headline.fit_transform(X_train['Headline'])
test_headline_hvec = hvec_headline.transform(X_test['Headline'])

lr = LogisticRegression()
lr.fit(train_headline_hvec, y_train)
lr.score(test_headline_hvec, y_test)

0.7313095333386697

In [33]:
#df_hv = pd.DataFrame(hvec.transform(X_test).todense(), index=[bodies.articleBody])  
#df_hv.head()

In [34]:
#df_hv.sum().sort_values(ascending=False).head(10)

In [35]:
#df_hv = df_hv.reindex_axis(df_hv.sum().sort_values(ascending=False).index, axis =1)
#df_hv

In [36]:
#what can i do with this 

# TF IDF 

In [37]:
#body
X_train = df_all[['articleBody']]
y_train = stances['Stance']
X_test = df_all[['articleBody']]
y_test = stances['Stance']

In [38]:
tvec_body = TfidfVectorizer(stop_words = 'english')

train_body_tvec = tvec_body.fit_transform(X_train['articleBody'])
test_body_tvec = tvec_body.transform(X_test['articleBody'])

lr = LogisticRegression()
lr.fit(train_body_tvec, y_train)
lr.score(test_body_tvec, y_test)

0.7313095333386697

In [39]:
#headline
X_train = df_all[['Headline']]
y_train = stances['Stance']
X_test = df_all[['Headline']]
y_test = stances['Stance']

In [40]:
tvec_headline = TfidfVectorizer(stop_words = 'english')

train_headline_tvec = tvec_headline.fit_transform(X_train['Headline'])
test_headline_tvec = tvec_headline.transform(X_test['Headline'])

lr = LogisticRegression()
lr.fit(train_headline_tvec, y_train)
lr.score(test_headline_tvec, y_test)

0.7313095333386697

In [41]:
#df_tfidf  = pd.DataFrame(tvec.transform(X_test).todense(),
                   #columns=tvec.get_feature_names(),
                   #index=[bodies.articleBody])
#df_tfidf.head()
#df.transpose().sort_values('bodies.articleBody', ascending=False).head(10).transpose()

In [42]:
#df_tfidf.sort_index(axis = 0, ascending=False).head()

In [43]:
#df_tfidf.sum().sort_values(ascending=False).head(10)

In [44]:
#df_tfidf = df_tfidf.reindex_axis(df_tfidf.sum().sort_values(ascending=False).index, axis =1)
#df_tfidf

## LDA

In [5]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # stemming
    #porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text_processed = [wordnet_lemmatizer.lemmatize(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- we're keeping our words distinct

In [46]:
df_all['articleBody'] = df_all['articleBody'].apply(text_process)

In [47]:
df_all['articleBody'].head()

0    [danny, boyle, directing, untitled, film, seth...
1    [danny, boyle, directing, untitled, film, seth...
2    [danny, boyle, directing, untitled, film, seth...
3    [danny, boyle, directing, untitled, film, seth...
4    [danny, boyle, directing, untitled, film, seth...
Name: articleBody, dtype: object

In [48]:
df_all['Headline'] = df_all['Headline'].apply(text_process)

In [49]:
df_all['Headline'].head()

0    [police, find, mass, graf, least, 15, body, ne...
1           [seth, rogen, play, apple, steve, wozniak]
2    [mexico, police, find, mass, grave, near, site...
3    [mexico, say, missing, student, found, first, ...
4          [new, io, 8, bug, delete, icloud, document]
Name: Headline, dtype: object

## Fit LDA Model

In [50]:
from gensim import corpora, models
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

np.random.seed(42)

## LDA Article Body

In [51]:
dictionary = corpora.Dictionary(df_all['articleBody'])

corpus = [dictionary.doc2bow(text) for text in df_all['articleBody']]

ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 20,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01) # only include topics above this probability threshold

In [52]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     8.472898        1       1 -0.226881  0.019120
9      7.595188        1       2  0.112287  0.000434
2      7.311213        1       3  0.083544  0.021570
13     6.791381        1       4  0.108508 -0.080753
8      6.466242        1       5 -0.174062  0.042057
7      5.406466        1       6  0.187084  0.060082
14     5.100646        1       7 -0.112049 -0.049270
3      4.986451        1       8 -0.218529  0.084044
12     4.938642        1       9  0.117559 -0.017840
6      4.882612        1      10 -0.059642  0.100910
0      4.623846        1      11 -0.111726  0.080458
19     4.126513        1      12  0.143788  0.016085
1      4.123122        1      13 -0.024215 -0.294692
18     4.007403        1      14  0.105911 -0.023171
5      3.879550        1      15  0.103321  0.141865
4      3.818975        1      16  0.013768 -0.042435
16     3.808369        1      17  0.129779  0.044201
10     3.591860        1      18  0.102500  0.032529
11     3.399776        1      19 -0.109934 -0.261607
15     2.668844        1      20 -0.171012  0.126411, topic_info=     Category           Freq          Term          Total  loglift  logprob
term                                                                       
6     Default   44046.000000         apple   44046.000000  30.0000  30.0000
1574  Default   30443.000000         watch   30443.000000  29.0000  29.0000
2527  Default   22237.000000           kim   22237.000000  28.0000  28.0000
97    Default   28784.000000            al   28784.000000  27.0000  27.0000
1218  Default   46117.000000         video   46117.000000  26.0000  26.0000
818   Default   34001.000000           isi   34001.000000  25.0000  25.0000
861   Default   20626.000000         north   20626.000000  24.0000  24.0000
353   Default   24472.000000        police   24472.000000  23.0000  23.0000
1180  Default   16882.000000         foley   16882.000000  22.0000  22.0000
363   Default   17941.000000          shot   17941.000000  21.0000  21.0000
3992  Default   13909.000000         brown   13909.000000  20.0000  20.0000
210   Default  130181.000000          said  130181.000000  19.0000  19.0000
343   Default   26540.000000       islamic   26540.000000  18.0000  18.0000
914   Default   41793.000000         state   41793.000000  17.0000  17.0000
107   Default   17448.000000        border   17448.000000  16.0000  16.0000
2895  Default   10801.000000         ebola   10801.000000  15.0000  15.0000
944   Default   47036.000000             u   47036.000000  14.0000  14.0000
2644  Default   12495.000000          jong   12495.000000  13.0000  13.0000
1215  Default   22614.000000         syria   22614.000000  12.0000  12.0000
563   Default   20635.000000      american   20635.000000  11.0000  11.0000
296   Default   29728.000000           man   29728.000000  10.0000  10.0000
816   Default   19261.000000          iraq   19261.000000   9.0000   9.0000
279   Default   16631.000000          girl   16631.000000   8.0000   8.0000
803   Default   33140.000000         group   33140.000000   7.0000   7.0000
1621  Default   13615.000000       officer   13615.000000   6.0000   6.0000
1785  Default   10156.000000         haram   10156.000000   5.0000   5.0000
1737  Default   10093.000000          boko   10093.000000   4.0000   4.0000
2649  Default   10183.000000         korea   10183.000000   3.0000   3.0000
43    Default   11492.000000           job   11492.000000   2.0000   2.0000
240   Default   10498.000000            un   10498.000000   1.0000   1.0000
...       ...            ...           ...            ...      ...      ...
8168  Topic20     478.524811       malaria     479.475220   3.6215  -6.4029
3573  Topic20     594.336731        frisco     595.911865   3.6209  -6.1861
1075  Topic20    4736.736328        banksy    4924.301758   3.5847  -4.1105
2227  Topic20    1919.160278        artist    1968.9793

## LDA Headline

In [53]:
dictionary = corpora.Dictionary(list(df_all['Headline']))

corpus = [dictionary.doc2bow(text) for text in df_all['Headline']]

ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 20,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01) # only include topics above this probability threshold

In [54]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
7      7.175739        1       1  0.160565 -0.236766
15     6.986013        1       2 -0.055301  0.092987
9      6.434521        1       3 -0.047171  0.080033
6      6.159083        1       4  0.033375 -0.118654
5      5.968647        1       5  0.172359  0.133394
19     5.822701        1       6  0.031229  0.184130
13     5.576653        1       7  0.031246 -0.039357
12     5.277154        1       8  0.190933  0.063809
16     4.846416        1       9 -0.262180  0.068002
11     4.759069        1      10  0.150684  0.096401
10     4.680370        1      11 -0.265636  0.023132
0      4.676740        1      12 -0.080662  0.049703
14     4.424639        1      13  0.220810 -0.044855
2      4.332467        1      14 -0.112297 -0.026703
4      4.281420        1      15 -0.079351 -0.041128
8      4.159107        1      16 -0.078881 -0.186555
3      3.989267        1      17  0.038145 -0.165177
18     3.813822        1      18  0.042447  0.220686
1      3.670227        1      19 -0.152054 -0.093390
17     2.965941        1      20  0.061741 -0.059691, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
221   Default  2875.000000       state  2875.000000  30.0000  30.0000
69    Default  6749.000000         isi  6749.000000  29.0000  29.0000
436   Default  2292.000000         kim  2292.000000  28.0000  28.0000
217   Default  2870.000000     islamic  2870.000000  27.0000  27.0000
14    Default  2829.000000       apple  2829.000000  26.0000  26.0000
430   Default  2661.000000       foley  2661.000000  25.0000  25.0000
435   Default  2099.000000        jong  2099.000000  24.0000  24.0000
73    Default  3714.000000           u  3714.000000  23.0000  23.0000
18    Default  1731.000000       steve  1731.000000  22.0000  22.0000
431   Default  2402.000000       james  2402.000000  21.0000  21.0000
468   Default  1844.000000       haram  1844.000000  20.0000  20.0000
465   Default  1844.000000        boko  1844.000000  19.0000  19.0000
37    Default  1624.000000         job  1624.000000  18.0000  18.0000
437   Default  1779.000000          un  1779.000000  17.0000  17.0000
94    Default  2009.000000      leader  2009.000000  16.0000  16.0000
80    Default  2329.000000       watch  2329.000000  15.0000  15.0000
505   Default  1652.000000       brown  1652.000000  14.0000  14.0000
432   Default  2153.000000  journalist  2153.000000  13.0000  13.0000
504   Default  1580.000000       audio  1580.000000  12.0000  12.0000
56    Default  2210.000000     michael  2210.000000  11.0000  11.0000
452   Default  2631.000000       video  2631.000000  10.0000  10.0000
428   Default  1920.000000    american  1920.000000   9.0000   9.0000
467   Default  1490.000000        girl  1490.000000   8.0000   8.0000
98    Default  1155.000000   christian  1155.000000   7.0000   7.0000
97    Default  1155.000000        bale  1155.000000   6.0000   6.0000
15    Default  1193.000000        play  1193.000000   5.0000   5.0000
486   Default  1056.000000          al  1056.000000   4.0000   4.0000
180   Default  1605.000000    militant  1605.000000   3.0000   3.0000
506   Default  1395.000000    shooting  1395.000000   2.0000   2.0000
12    Default  1026.000000     student  1026.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1235  Topic20    54.135845      agrees    55.081535   3.5007  -5.4582
1519  Topic20    53.068871        pack    54.014561   3.5003  -5.4781
1515  Topic20    53.068871        brat    54.014561   3.5003  -5.4781
1041  Topic20    52.950623   cawthorne    53.896313   3.5003  -5.4804
1105  Topic20    47.330612      briton    48.276302   3.4982  -5.5926
1987  Topic20    46.501282      timing    47.446972   3.4978  -5.6102
1988  Topic20    46.501282    verifies    47.446972   3.4978 

### preprocessing for Latent Semantic Analysis

In [55]:
#need this cleaner to make vectorizers below work....idk why

In [56]:
import re

In [57]:
def cleaner(text):
    text = re.sub('&#39;','',text).lower()
    text = re.sub('<br />','',text)
    text = re.sub('<.*>.*</.*>','', text)
    text = re.sub('\\ufeff', '', text)
    text = re.sub('[\d]','',text)
    text = re.sub('[^a-z ]','',text)
    
    return text

<input>:6: DeprecationWarning: invalid escape sequence \d
<input>:6: DeprecationWarning: invalid escape sequence \d
<input>:6: DeprecationWarning: invalid escape sequence \d
<ipython-input-57-6de38bea8180>:6: DeprecationWarning: invalid escape sequence \d
  text = re.sub('[\d]','',text)


In [58]:
df_all['articleBody'] = df_all['articleBody'].map(str)
df_all['articleBody'] = df_all['articleBody'].apply(cleaner)

In [59]:
df_all['articleBody'].iloc[1]

'danny boyle directing untitled film seth rogen eyed play apple co founder steve wozniak sony steve job biopic danny boyle directing untitled film based walter isaacson book adapted aaron sorkin one anticipated biopics recent year negotiation yet begun even clear rogen official offer producer scott rudin guymon casady mark gordon set sight talent talk course may naught christian bale actor play job still midst closing deal source say dealmaking process sensitive stage insider say boyle flying los angeles meet actress play one female lead assistant job insider say jessica chastain one actress meeting list wozniak known woz co founded apple job ronald wayne first met job worked atari later responsible creating early apple computer'

In [60]:
df_all['Headline'] = df_all['Headline'].map(str)
df_all['Headline'] = df_all['Headline'].apply(cleaner)

In [61]:
df_all.head()
#why does this change my text processing back 

,Headline,Body ID,Stance,articleBody
0,police find mass graf least body near mexico ...,712,unrelated,danny boyle directing untitled film seth rogen...
1,seth rogen play apple steve wozniak,712,discuss,danny boyle directing untitled film seth rogen...
2,mexico police find mass grave near site stude...,712,unrelated,danny boyle directing untitled film seth rogen...
3,mexico say missing student found first mass graf,712,unrelated,danny boyle directing untitled film seth rogen...
4,new io bug delete icloud document,712,unrelated,danny boyle directing untitled film seth rogen...


In [62]:
df_all.to_csv('./preprocessed.csv')

In [21]:
df_all = pd.read_csv('./preprocessed.csv')

In [22]:
df_all.head()

,Unnamed: 0,Headline,Body ID,Stance,articleBody
0,0,police find mass graf least body near mexico ...,712,unrelated,danny boyle directing untitled film seth rogen...
1,1,seth rogen play apple steve wozniak,712,discuss,danny boyle directing untitled film seth rogen...
2,2,mexico police find mass grave near site stude...,712,unrelated,danny boyle directing untitled film seth rogen...
3,3,mexico say missing student found first mass graf,712,unrelated,danny boyle directing untitled film seth rogen...
4,4,new io bug delete icloud document,712,unrelated,danny boyle directing untitled film seth rogen...


### tfidf vectorizer

In [23]:
tfidf_vector = TfidfVectorizer(min_df = 5, stop_words='english')

articleBody_matrix_sparse = tfidf_vector.fit_transform(df_all['articleBody'])
articleBody_matrix_sparse

<49972x18408 sparse matrix of type '<class 'numpy.float64'>'
	with 6480420 stored elements in Compressed Sparse Row format>

In [24]:
articleBody_df_tfdf = pd.DataFrame(articleBody_matrix_sparse.toarray(),
                                 index = df_all.index,
                                 columns = tfidf_vector.get_feature_names())

In [65]:
articleBody_df_tfdf.head()

,aafia,aamaq,aamir,aan,aapl,aaron,aback,abadam,abadi,abagnale,...,zobl,zone,zoning,zoology,zor,zubayr,zubeyr,zuckerberg,zulu,zxycaeuwb
0,0.0,0.0,0.0,0.0,0.0,0.07737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.07737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.07737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.07737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.07737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
headline_matrix_sparse = tfidf_vector.fit_transform(df_all['Headline'])
headline_matrix_sparse

<49972x2736 sparse matrix of type '<class 'numpy.float64'>'
	with 388147 stored elements in Compressed Sparse Row format>

In [26]:
headline_df_tfdf = pd.DataFrame(headline_matrix_sparse.toarray(),
                                 index = df_all.index,
                                 columns = tfidf_vector.get_feature_names())

In [68]:
headline_df_tfdf.head()

,aaron,abandoned,abdel,abdi,abducted,abducting,abdul,abdullah,abort,aborted,...,younger,youngest,youtube,ypg,yum,zack,zehaf,zeppelin,zero,zhejiang
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Truncated SVD

In [27]:
from sklearn.decomposition import TruncatedSVD

In [28]:
n_components = 100
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i) for i in range(n_components)]

In [71]:
#lsa = TruncatedSVD(n_components=100, n_iter=100)
#lsa.fit(articleBody_df_tfdf)

In [72]:
#lsa.components_[0]

In [73]:
#terms = vectorizer.get_feature_names()
#for i, comp in enumerate(lsa.components_):
    #termsInComp = zip (terms, comp)
    #sortedTerms = sorted(termsInComp, key=lambda X: x[1], reverse = True)[:10]
    #print "Word %d:" % i
    #for term in sortedTerms:
        #print term[0]
        #print " "

In [29]:
articleBody_svd_matrix = SVD.fit_transform(articleBody_df_tfdf)
articleBody_svd_matrix

array([[ 0.09533844,  0.20917701, -0.02623359, ...,  0.00920228,
        -0.01856594, -0.00950194],
       [ 0.09533844,  0.20917701, -0.02623359, ...,  0.00920228,
        -0.01856594, -0.00950194],
       [ 0.09533844,  0.20917701, -0.02623359, ...,  0.00920228,
        -0.01856594, -0.00950194],
       ...,
       [ 0.10926746, -0.00405863, -0.01237419, ..., -0.00385819,
         0.001803  , -0.0149248 ],
       [ 0.17433644,  0.51243698, -0.07441508, ..., -0.00582191,
         0.00681073,  0.05076078],
       [ 0.16292307,  0.66241454, -0.12880881, ..., -0.02244739,
         0.00879433,  0.06297349]])

In [30]:
headline_svd_matrix = SVD.fit_transform(headline_df_tfdf)
headline_svd_matrix

array([[ 0.01147479,  0.00935641,  0.00474512, ...,  0.02327761,
        -0.04932604,  0.03435446],
       [ 0.00723131,  0.05075058, -0.00227545, ...,  0.04016022,
         0.01293309,  0.06570908],
       [ 0.01427152,  0.0072916 ,  0.00614599, ...,  0.01756991,
        -0.02915877,  0.00694797],
       ...,
       [ 0.01233547,  0.00230589,  0.00526141, ...,  0.00475152,
        -0.05665834, -0.02388791],
       [ 0.01735225,  0.01500701,  0.00583595, ...,  0.01591702,
        -0.02312003,  0.00565127],
       [ 0.00979544,  0.01030163,  0.00912464, ...,  0.03620044,
        -0.00023186, -0.01525898]])

In [76]:
articleBody_svd_matrix.shape

(49972, 100)

In [77]:
#search_term = "islam"

#use headline to compare to the boody 
#make a list forbody and article and loop thru each row 

In [78]:
#search_term_vec = tfidf_vector.transform([search_term])

In [79]:
#search_term_svd = SVD.transform(search_term_vec)

In [80]:
#search_term_svd

## Cosine Similarity

In [81]:
#cosine_similarities = articleBody_svd_matrix.dot(search_term_svd.T).ravel()
#do for 1 row with another row --> feature 

In [82]:
#cosine_similarities.shape

In [83]:
#cosine_similarities

In [84]:
#cosine_similarities.argsort()[:-6:-1]

In [85]:
#cosine_similarities_headline = headline_svd_matrix.dot(search_term_svd.T).ravel()

In [86]:
from gensim import corpora, models
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [87]:
#from nltk.corpus import stopwords
#def split_into_words(documents):
    #docs = []
    #for doc in documents:
        #text_processed = doc.split(' ')
        #text_processed = list(filter(lambda a: a != '', text_processed))
        #text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
        #docs.append(text_processed)
    #return docs

In [88]:
#texts = split_into_words(df_all['articleBody'])

## Cosine Similarity

In [31]:
import scipy.spatial as sp

In [32]:
#distances = 1 - sp.distance.cdist(articleBody_svd_matrix, headline_svd_matrix, 'cosine')

In [33]:
distances = []
for i, body in enumerate(articleBody_svd_matrix): 
    distances.append(sp.distance.cdist([body], [headline_svd_matrix[i]], 'cosine')[0][0])

In [45]:
df_all['distances'] = pd.Series(distances)

In [117]:
df_all.shape

(49972, 6)

### Eliminating Nulls

In [ ]:
#df_fix = pd.DataFrame(df_all['distances'], df_all['Body ID'], df_all['Stance'])

In [95]:
df_fix.T.head()

Body ID,712,712,712,712,712,712,712,712,712,712,...,797,74,135,175,553,464,362,915,407,1066
Stance,,,,,,,,,,,,,,,,,,,,,
unrelated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
discuss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unrelated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unrelated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unrelated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#2 null values 
df_all.shape

(49972, 5)

In [35]:
df_all = df_all.dropna(axis=0)

#why does this change my df to not a df 

In [98]:
df_all.shape

(49970, 5)

In [36]:
df_all = stances.merge(bodies, on = 'Body ID')

In [46]:
df_all.head()

,Headline,Body ID,Stance,articleBody,stance_dummy,distances
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3,0.848807
1,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss,Danny Boyle is directing the untitled film\n\n...,2,0.954702
2,Mexico police find mass grave near site 43 stu...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3,0.841187
3,Mexico Says Missing Students Not Found In Firs...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3,0.768969
4,New iOS 8 bug can delete all of your iCloud do...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3,1.033323


In [101]:
df_fix.head()

Stance,unrelated,discuss,unrelated,unrelated,unrelated,discuss,discuss,unrelated,unrelated,unrelated,...,agree,discuss,agree,discuss,agree,agree,agree,agree,discuss,discuss
Body ID,,,,,,,,,,,,,,,,,,,,,
712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/anaconda3/envs/dsi/lib/python3.6/site-packages/_pytest/mark/structures.py:284: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  _marks = attr.ib(convert=list)


KeyError: 'distances'

In [103]:
df_fix = df_fix.dropna(axis=0)

In [104]:
df_fix.shape

(0, 49972)

In [105]:
df_all[df_all['distances'].isnull()==True]

KeyError: 'distances'

### Adding labels to stances

In [37]:
from sklearn import preprocessing

In [38]:
le = preprocessing.LabelEncoder()
le.fit(df_all['Stance'])
print("Classes:", list(le.classes_))
stance_label = le.transform(df_all['Stance']) 


Classes: ['agree', 'disagree', 'discuss', 'unrelated']


In [39]:
stance_label

array([3, 2, 3, ..., 0, 2, 2])

In [40]:
df_all['stance_dummy'] = pd.Series(stance_label)

In [41]:
df_all.head()

,Headline,Body ID,Stance,articleBody,stance_dummy
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3
1,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss,Danny Boyle is directing the untitled film\n\n...,2
2,Mexico police find mass grave near site 43 stu...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3
3,Mexico Says Missing Students Not Found In Firs...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3
4,New iOS 8 bug can delete all of your iCloud do...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3


In [42]:
stance_dummy = pd.get_dummies(df_all['Stance'], drop_first=False)
stance_dummy.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,1,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [48]:
df_fix = pd.DataFrame([df_all['distances'], df_all['stance_dummy']]).T
df_fix = df_fix.dropna(axis=0)
df_fix.head()

,distances,stance_dummy
0,0.848807,3.0
1,0.954702,2.0
2,0.841187,3.0
3,0.768969,3.0
4,1.033323,3.0


In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X = df_fix['distances'].reshape(df_fix['distances'].shape[0], 1)
y = df_fix['stance_dummy']

X_train, X_test, y_train, y_test = train_test_split(X, y)

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [51]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((37477, 1), (12493, 1), (37477,), (12493,))

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
np.unique(y_train)

### ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_probas = lr.predict_proba(X_train)
y_true = y_train

import scikitplot as skplt
import matplotlib.pyplot as plt

skplt.metrics.plot_roc(y_true, y_probas, figsize=(7,7))
plt.show()

In [ ]:
y_true.unique(), y_true.shape

In [ ]:
np.unique(y_probas), y_probas.shape

In [ ]:
y_probas

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

In [ ]:
lr = LogisticRegression()
lr.fit(X_resampled, y_resampled)
lr.score(X_resampled, y_resampled)

# Model with cvec body, cvec head, and cosine similarity

In [ ]:
cvec_vector = CountVectorizer(stop_words='english')

cvec_articleBody_matrix_sparse = cvec_vector.fit_transform(df_all['articleBody'])
cvec_articleBody_matrix_sparse

In [ ]:
cvec_body_array = cvec_articleBody_matrix_sparse.toarray()

In [ ]:
df_cvec = pd.DataFrame(cvec_body_array)

In [ ]:
df_cvec.head()

In [ ]:
cvec_vector = CountVectorizer(stop_words='english')

cvec_headline_matrix_sparse = cvec_vector.fit_transform(df_all['Headline'])

cvec_headline_matrix_sparse

In [ ]:
cvec_vector.get_feature_names

In [ ]:
cvec_headline_df = pd.DataFrame(cvec_headline_matrix_sparse.todense(),
                       columns = cvec_vector.get_feature_names())


In [ ]:
cvec_headline_df.shape

In [ ]:
df_fix.shape

In [ ]:
df_headline_fix = pd.concat([df_fix, cvec_headline_df], axis=1)

In [ ]:
df_headline_fix.fillna(value=0, inplace=True)

In [ ]:
features = [col for col in df_headline_fix.columns if col != 'stance_dummy']

In [ ]:
X = df_headline_fix[features]
y = df_headline_fix['stance_dummy']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

## GloVe


In [69]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

max_sequence_length = 50   # max number of words to consider in a review (i.e. the first 50...)
max_num_words = 4000       # maximum number of words to include in the vocabulary (i.e. top 10000 only)

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words=max_num_words)

# Fit the tokenizer, i.e. learn the vocab and id the most frequently occuring words
tokenizer.fit_on_texts(df_all['Headline'].values)

# Turn our texts to sequences of word indices
sequences = tokenizer.texts_to_sequences(df_all['Headline'].values)

# Save the look-up dictionary for words to indices (will need this later)
word_index = tokenizer.word_index

# Pad out our sequences by prepending zeros to all of our text sequences
padded_sequences_headline = pad_sequences(sequences, maxlen=max_sequence_length)

print(f'Found {len(word_index)} unique tokens/words')
print(f'The maximum word index is {padded_sequences.max()}')

Found 3879 unique tokens/words
The maximum word index is 24999


In [70]:
padded_sequences_headline.shape

(49972, 50)

In [71]:
padded_sequences_headline[0,:]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  248,  540,
        133,  361,   14,   12,  173, 2481, 2482,  206,  118, 1223, 1341,
        674,  143, 2483,   13,  248, 2484], dtype=int32)

In [79]:
max_sequence_length = 200   # max number of words to consider in a review (i.e. the first 50...)
max_num_words = 4000       # maximum number of words to include in the vocabulary (i.e. top 10000 only)

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words=max_num_words)

# Fit the tokenizer, i.e. learn the vocab and id the most frequently occuring words
tokenizer.fit_on_texts(df_all['articleBody'].values)

# Turn our texts to sequences of word indices
sequences = tokenizer.texts_to_sequences(df_all['articleBody'].values)

# Save the look-up dictionary for words to indices (will need this later)
word_index = tokenizer.word_index

# Pad out our sequences by prepending zeros to all of our text sequences
padded_sequences_articleBody = pad_sequences(sequences, maxlen=max_sequence_length)

print(f'Found {len(word_index)} unique tokens/words')
print(f'The maximum word index is {padded_sequences.max()}')

Found 27427 unique tokens/words
The maximum word index is 24999


In [80]:
padded_sequences_articleBody.shape

(49972, 200)

In [81]:
padded_sequences_articleBody[0,:]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1961, 1800,    8,    1,  473, 2509, 1702,    8,   83,    2,  688,
         45,  528, 1102,  524, 2033,    5,  524,  381, 1542, 1961, 1800,
          8,    1,  473,  291,   10, 2300, 3475,  912,    6,   19, 1626,
        935,   42,    8,   40,    4,    1,  169, 2848,    5,  312,  133,
       1155,   17,   25,  317, 3047,    6,  186,   25,  115,  521,   64,
       1702,   20,   24,  145,  988,   29,    1, 3300,   87, 3533,    6,
        565, 2432,   87,   17,  314,   50,   10,    1,    6,   26,    5,
        729,    4,  996,   30,  110,   57,   23,   11,   18,  868,  564,
          1,  677,   32,    8,    2,  688,  381,    8,  142,    5,    1,
          4,   16,  645,  246,  132,    7, 1624,    8,    5,    3, 1772,
        132, 1800,   38,    8, 1287,    2, 2897, 30

In [82]:
padded_sequences = np.concatenate((padded_sequences_headline, padded_sequences_articleBody), axis=1)
padded_sequences.shape

(49972, 250)

In [83]:
df_all['stance_dummy'].shape

(49972,)

In [84]:
# Train, test split, baby...
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df_all['stance_dummy'], test_size=0.33, random_state=42)

X_train.shape, y_train.shape

((33481, 250), (33481,))

In [85]:
import numpy as np

glove_dir = './glove.6B'

embeddings_index = {}

with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print(f'Found {len(embeddings_index)} word vectors')

Found 400000 word vectors


In [86]:
embedding_dim = 100

embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, index in word_index.items():
    if index < max_num_words:
        # Using "get" with the dict is a safe way to avoid missing key errors
        embedding_vector = embeddings_index.get(word) 
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [87]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(max_num_words, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(20))
model.add(Dense(1, activation='sigmoid'))

In [88]:
model.layers[0].set_weights([embedding_matrix])

In [89]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          400000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 20)                9680      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 409,701
Trainable params: 409,701
Non-trainable params: 0
_________________________________________________________________


In [94]:
history = model.fit(X_train, y_train, epochs=5, batch_size=300, validation_data=(X_test, y_test))

ValueError: Error when checking input: expected embedding_3_input to have shape (200,) but got array with shape (250,)

In [ ]:
# What's baseline?# What's 
np.unique(y_train, return_counts=True)[1]/len(y_train)

In [ ]:
X_test[0:1,:].shape

In [ ]:
model.predict(X_test[0:1,:])

### body

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(df_all['articleBody'], size=100, window=5, min_count=1, workers=4)
body_w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [ ]:
df_body_w2v = pd.DataFrame(body_w2v)
df_body_w2v.head()

## Model with word2vec articleBody + headline 

In [ ]:
#from tabulate import tabulate
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

# TRAIN_SET_PATH = "20ng-no-stop.txt"
# TRAIN_SET_PATH = "r52-all-terms.txt"
TRAIN_SET_PATH = "r8-no-stop.txt"

GLOVE_6B_50D_PATH = "glove.6B.50d.txt"
GLOVE_840B_300D_PATH = "glove.840B.300d.txt"
encoding="utf-8"

In [ ]:
ls = []
sentences = df_glove.split(".")
for i in sentences:
    ls.append(i.split())
model = Word2Vec(df_glove, min_count=1, size = 4)
words = list(model.wv.vocab)
print(words)
vectors = []
for word in words:
    vectors.append(model[word].tolist())
data = np.array(vectors)
data

In [ ]:
df_w2v_fix = pd.concat([df_body_w2v, df_w2v], axis=1)
df_w2v_fix.head()

In [ ]:
df_w2v_fix.shape

In [ ]:
df_fix['Unnamed 0'].shape

In [ ]:
X = df_w2v_fix
y = df_fix['Unnamed 0'] #need to create binary for classes #predict the prob of each class and highest use as predictions

X_train, X_test, y_train, y_test = train_test_split(X, y)